# Creating a Model 

In [1]:
import importlib, os, utils, urllib
from datetime import datetime, timedelta
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


2023-05-08 00:00:21.372645: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 00:00:21.755925: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 00:00:21.758714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 00:00:23.812991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
importlib.reload(utils)
all_zones_df = utils.clean_raw_webscraper_data('output_data/All_Zones_2021-22_Season_reports_data.csv')
all_zones_df = utils.add_weather_to_reports(all_zones_df)
all_zones_df.head()

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,bottom_line_text,problem_type_text,forecast_discussion_text,combined_text,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2022-04-23,east slopes north,3.0,3.0,3.0,2.0,avalanche danger minimal morning quickly spike...,expect warmest temperature almost week enough ...,cornice fall snow blanket cornice rock face ne...,avalanche danger minimal morning quickly spike...,7.6,-0.9,15.9,0.0,225.3,1024.0,0.0
1,2022-04-23,west slopes central,3.0,3.0,3.0,2.0,avalanche danger minimal morning quickly spike...,expect warmest temperature almost week enough ...,cornice fall example cornice failure glide ava...,avalanche danger minimal morning quickly spike...,9.9,4.6,15.4,0.0,242.4,1026.2,0.0
2,2022-04-23,west slopes south,3.0,3.0,3.0,2.0,weather look beautiful could create dangerous ...,mostly clear sky help surface freeze solid nig...,cornice fall example cornice slope large overh...,weather look beautiful could create dangerous ...,9.9,4.4,16.3,0.0,279.4,1026.0,0.0
3,2022-04-23,east slopes central,2.0,2.0,2.0,2.0,spring shed underway multi day warming trend p...,warmup wet loose avalanche expected next day a...,cornice fallcornice failure glide avalanche ex...,spring shed underway multi day warming trend p...,10.6,5.3,16.3,0.0,282.1,1025.4,0.0
4,2022-04-23,snoqualmie pass,3.0,3.0,3.0,2.0,weather look beautiful however sunshine warm t...,solid overnight freeze limit loose wet hazard ...,cornice fall example block cornice trigger fel...,weather look beautiful however sunshine warm t...,10.2,4.8,16.3,0.0,274.1,1025.8,0.0


In [3]:
data = all_zones_df.drop(columns=['bottom_line_text', 'problem_type_text', 'forecast_discussion_text'])

data['combined_text'] = data['combined_text'].astype(str)
data['combined_text'] = data['combined_text'].apply(lambda x: word_tokenize(x.lower()))
data.rename(columns={'combined_text': 'tokens'}, inplace=True)

data.head()

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,tokens,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2022-04-23,east slopes north,3.0,3.0,3.0,2.0,"[avalanche, danger, minimal, morning, quickly,...",7.6,-0.9,15.9,0.0,225.3,1024.0,0.0
1,2022-04-23,west slopes central,3.0,3.0,3.0,2.0,"[avalanche, danger, minimal, morning, quickly,...",9.9,4.6,15.4,0.0,242.4,1026.2,0.0
2,2022-04-23,west slopes south,3.0,3.0,3.0,2.0,"[weather, look, beautiful, could, create, dang...",9.9,4.4,16.3,0.0,279.4,1026.0,0.0
3,2022-04-23,east slopes central,2.0,2.0,2.0,2.0,"[spring, shed, underway, multi, day, warming, ...",10.6,5.3,16.3,0.0,282.1,1025.4,0.0
4,2022-04-23,snoqualmie pass,3.0,3.0,3.0,2.0,"[weather, look, beautiful, however, sunshine, ...",10.2,4.8,16.3,0.0,274.1,1025.8,0.0


In [4]:
all_zones_df.to_csv('output_data/all_zones_all_data.csv', index=False)

In [31]:
# Training the word to vec model:
model = Word2Vec(sentences=data['tokens'], vector_size=100, window=5, min_count=1, workers=4)
model.save('models/first_word2vec.model')

# Create word embeddings lookup dictionary
word_embeddings = {}

for word in model.wv.index_to_key:
    word_embeddings[word] = model.wv[word]


In [48]:
# Convert text data to sequences of word embeddings
max_sequence_length = max(data['tokens'].apply(len))

def convert_text_to_embeddings(text):
    embeddings = []
    
    for word in text:
        if word in word_embeddings:
            embeddings.append(word_embeddings[word])
        else:
            embeddings.append([0.0] * embedding_dimension)
    
    return embeddings

sequences = data['tokens'].apply(convert_text_to_embeddings)

# Update the padding step
padded_sequences = pad_sequences(sequences.apply(lambda x: [elem[:embedding_dimension] for elem in x]),
                                maxlen=max_sequence_length, padding='post')

data

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,tokens,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2022-04-23,east slopes north,3.0,3.0,3.0,2.0,"[avalanche, danger, minimal, morning, quickly,...",7.6,-0.9,15.9,0.0,225.3,1024.0,0.0
1,2022-04-23,west slopes central,3.0,3.0,3.0,2.0,"[avalanche, danger, minimal, morning, quickly,...",9.9,4.6,15.4,0.0,242.4,1026.2,0.0
2,2022-04-23,west slopes south,3.0,3.0,3.0,2.0,"[weather, look, beautiful, could, create, dang...",9.9,4.4,16.3,0.0,279.4,1026.0,0.0
3,2022-04-23,east slopes central,2.0,2.0,2.0,2.0,"[spring, shed, underway, multi, day, warming, ...",10.6,5.3,16.3,0.0,282.1,1025.4,0.0
4,2022-04-23,snoqualmie pass,3.0,3.0,3.0,2.0,"[weather, look, beautiful, however, sunshine, ...",10.2,4.8,16.3,0.0,274.1,1025.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2021-12-27,east slopes north,2.0,2.0,1.0,1.0,"[extremely, cold, temperature, keep, avalanche...",-22.1,-26.2,-18.6,0.4,283.6,1017.6,0.0
1176,2021-12-27,stevens pass,2.0,2.0,2.0,2.0,"[trigger, avalanche, recent, snow, slope, stee...",-9.1,-12.1,-6.7,0.4,86.1,1009.9,0.0
1177,2021-12-27,east slopes central,2.0,2.0,1.0,1.0,"[cold, temperature, moderate, wind, speed, kee...",-10.2,-13.6,-7.5,0.3,96.8,1010.1,0.0
1178,2021-12-27,west slopes central,2.0,2.0,2.0,2.0,"[significant, new, snowfall, last, day, create...",-10.1,-12.9,-7.4,0.4,62.2,1011.3,0.0


: 

In [ ]:

# Define LSTM model
embedding_dimension = 100
num_lstm_units = 64
num_classes = 2

model = Sequential()
model.add(Embedding(input_dim=len(word_embeddings), output_dim=embedding_dimension,
                    input_length=max_sequence_length, trainable=False))
model.add(LSTM(units=num_lstm_units))
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Split data into training and validation sets
train_size = int(len(data) * 0.8)
X_train = padded_sequences[:train_size]
y_train = data['overall_risk'].values[:train_size]
X_val = padded_sequences[train_size:]
y_val = data['overall_risk'].values[train_size:]

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Example prediction
new_text = ['Slabs of snow are sitting on top of a weak layer of sugary snow.']
new_tokens = word_tokenize(new_text[0].lower())
new_sequence = convert_text_to_embeddings(new_tokens)
new_padded_sequence = pad_sequences([new_sequence], maxlen=max_sequence_length, padding='post')

prediction = model.predict(new_padded_sequence)
predicted_label = prediction.argmax(axis=-1)[0]
print(f"Predicted label: {predicted_label}")

## Use the Most Similar word in the final app:

In [32]:
word = 'human'
model.wv.most_similar(word)

[('skier', 0.9070470929145813),
 ('naturally', 0.9053963422775269),
 ('triggered', 0.8907420635223389),
 ('remotely', 0.8899236917495728),
 ('isolated', 0.867769181728363),
 ('suggests', 0.8642491102218628),
 ('primed', 0.8624805212020874),
 ('rare', 0.8488191366195679),
 ('traveler', 0.845313310623169),
 ('avalanches', 0.8452948331832886)]